# Image classification with transfer learning

**Data Pre-processing**

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

#Importing dataset from CIFAR
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split


# Load CIFAR-10 dataset and print shapes
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("x_test shape:", x_test.shape)
print("y_test shape:", y_test.shape)

x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
x_test shape: (10000, 32, 32, 3)
y_test shape: (10000, 1)


**Pre-processing classes (y)**

In [10]:
# convert classes into categories with one hot encoding and check shape.
from tensorflow.keras.utils import to_categorical

num_classes = 10

y_train_cat = to_categorical(y_train, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

print("y_train_cat shape:", y_train_cat.shape)
print("y_test_cat shape:", y_test_cat.shape)

y_train_cat shape: (50000, 10)
y_test_cat shape: (10000, 10)


**Pre-processing images**

to make them compatible with the shapes and scales in our base model (MobileNetV2)


In [6]:
# This Keras function is specifically designed to make images compatible with the MobileNetV2 model.
# It involves scaling and shifting pixel values to [-1, 1], as the model was trained on.

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

# Preprocess your data
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)


In [7]:
# Since we are using a Transfer learning technique, we need to merge features and labels into datasets, so that they can be processed correctly at a later stage.

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))

# Create the base model from the pre-trained convnets

In [8]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE = (32,32,3)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')



#Freeze the convolutional base to prevent the weights from being updated during training.
base_model.trainable = False



/tmp/ipython-input-788324976.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,


### LOOKING GOOD UNTIL HERE

In [ ]:



# get a batch of images from our dataset and pass it through base model to get feature maps
image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)
print(f"Feature batch shape: {feature_batch.shape}")

# Apply Global Average Pooling to convert feature maps to vectors
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(f"Feature batch average shape: {feature_batch_average.shape}")



In [ ]:
# Add a classification head.
# In our case, it will be 10 neurons dense since this is the number of categories.
# Using Softmax as the activation function since this is the ideal choice for classification problems.
prediction_layer = tf.keras.layers.Dense(10, activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(f"Prediction batch shape: {prediction_batch.shape}")



In [ ]:
#data_augmentation = tf.keras.Sequential([
#    tf.keras.layers.RandomFlip('horizontal'),
#    tf.keras.layers.RandomRotation(0.2),
#])


inputs = tf.keras.Input(shape=(32, 32, 3))
#x = data_augmentation(x_train)
x = preprocess_input(x_train)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [ ]:
##block of code used to clear keras sessions during development

#from keras.backend import clear_session
#clear_session()